In [32]:
import csv 
import pandas as pd
import numpy as np
from datetime import datetime 
import re
from unidecode import unidecode

In [33]:
#Diretórios
DIR_ESCOLAS = r'..\arquivos csv\escolas.csv'
DIR_MATERIAL_DIDATICO = r'..\arquivos csv\material_didatico.csv'
DIR_SUB_PREFEITURAS = r'..\arquivos csv\subprefeituras.csv'


# Importações

In [34]:
#Importações
escolas = pd.read_csv(DIR_ESCOLAS)
material_didatico = pd.read_csv(DIR_MATERIAL_DIDATICO)
sub_prefeituras = pd.read_csv(DIR_SUB_PREFEITURAS)

In [35]:
escolas.head(2)
escolas.drop_duplicates(inplace=True)

In [36]:
material_didatico.head(2)
material_didatico.drop_duplicates(inplace=True)

In [37]:
sub_prefeituras.head(2)
sub_prefeituras.drop_duplicates(inplace=True)

# Merge

In [38]:
#Quantidade de material didático (md) à ser entregue
escolas_quantidade_md = pd.merge(escolas, material_didatico, how='left', on='id')
escolas_quantidade_md.head(2)

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326",20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202",121


In [39]:
#Define os tipos de escolas

def define_tipo(x):
    if 'EM' in x.upper() or 'E.M' in x.upper():
        return 'EM'
    elif 'CIEP' in x.upper() or 'CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA' in x.upper():
        return 'CIEP'
    elif 'ESCOLA' in x.upper():
        return 'ESCOLA'
    else:
        None
    
escolas_quantidade_md['tipo'] = escolas_quantidade_md['Escolas_Postos'].apply(lambda x: define_tipo(x))
escolas_quantidade_md.head(2)

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,Quantidade,tipo
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326",20,CIEP
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202",121,EM


# Tratamentos

In [40]:
# Converte todos os valores strings para UPPER CASE
escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: x.upper() if isinstance(x, str) else x)
sub_prefeituras = sub_prefeituras.applymap(lambda x: x.upper() if isinstance(x, str) else x)
# Remove acentuações
escolas_quantidade_md = escolas_quantidade_md.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
sub_prefeituras = sub_prefeituras.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
# Padronizando nomes das colunas em snake_case
column_mapping = {col: col.strip().lower().replace(" ", "_") for col in escolas_quantidade_md.columns}
escolas_quantidade_md.rename(columns=column_mapping, inplace=True)
escolas_quantidade_md.rename(columns={'endereço':'endereco'},inplace=True)
escolas_quantidade_md.head(2)

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,tipo
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,"-22,880888","-43,225326",20,CIEP
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",121,EM


In [41]:
# Função para substituir palavras em texto da coluna 'endereco'
def substituir_palavras(texto, substituicoes):
    for palavra, substituta in substituicoes.items():
        texto = texto.replace(palavra, substituta)
    return texto

substituicoes_endereco = {
    '.': '',
    ',': '',
    'ESTR ': 'ESTRADA ',
    'JR ': 'JUNIOR ',
    'AV ': 'AVENIDA ',
    'MAL ': 'MARECHAL ',
    'ENG ': 'ENGENHEIRO ',
    'SD ': 'SOLDADO ',
    'PCA ': 'PRACA ',
    'ALM ': 'ALMIRANTE ',
    'SEN ': 'SENADOR ',
    'CONJ ': 'CONJUNTO ',
}

# Define um padrão para encontrar "R" como uma palavra independente em endereco para substituição
padrao_r = r'\bR\b'

# Aplica as funções de substituição à coluna endereco
escolas_quantidade_md['endereco'] = escolas_quantidade_md['endereco'].apply(substituir_palavras, args=(substituicoes_endereco,))
escolas_quantidade_md['endereco'] = escolas_quantidade_md['endereco'].str.replace(padrao_r, 'RUA', regex=True)

# Função para extrair complementos do endereço
def extrair_complemento(endereco):
    complementos = re.findall(r'\b(BLOCO|CONJUNTO)\s+(\S+)\b', endereco, flags=re.IGNORECASE)
    if complementos:
        return ','.join([f'{comp[0]} {comp[1]}' for comp in complementos])
    else:
        return "SEM COMPLEMENTO"

# Aplica a função de extração e cria uma nova coluna 'complemento'
escolas_quantidade_md['complemento'] = escolas_quantidade_md['endereco'].apply(extrair_complemento)

# Função para extrair números do endereço
def extrair_numero(endereco):
    numeros_hifen = re.findall(r'\b\d+-\d+\b', endereco)
    numeros_km = re.findall(r'KM\s*\d+',endereco)
    padrao_sem_numeros = re.findall(r'S/No|S/Ndeg|S/N', endereco)

    if numeros_hifen:
        return '-'.join(numeros_hifen)
    elif numeros_km:
        return numeros_km[-1]
    elif padrao_sem_numeros:
        return 0
    else:
        numeros = re.findall(r'\d+', endereco)
        return int(numeros[-1]) if numeros else 0

# Aplica a função de extração e cria uma nova coluna 'numero_rua'
escolas_quantidade_md['numero_rua'] = escolas_quantidade_md['endereco'].apply(extrair_numero)

# Função para remover números e variações do endereço
def remover_numeros_variacoes(endereco, numeros_rua):
    if isinstance(numeros_rua, str):
        endereco = endereco.replace(numeros_rua, '')
    padrao = r'\d+$|S/No|S/Ndeg|S/N|-'
    endereco_limpo = re.sub(padrao, '', endereco).strip()
    return endereco_limpo

# Aplica a função de remoção de números do endereço
escolas_quantidade_md['endereco'] = escolas_quantidade_md.apply(lambda row: remover_numeros_variacoes(row['endereco'], row['numero_rua']), axis=1)
escolas_quantidade_md.head(2)

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,tipo,complemento,numero_rua
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,880888","-43,225326",20,CIEP,SEM COMPLEMENTO,0
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,"-22,889574","-43,236202",121,EM,SEM COMPLEMENTO,187


# Mergeando em Subprefeituras


In [42]:
escolas_quantidade_md = escolas_quantidade_md.merge(sub_prefeituras, left_on='bairro', right_on='nome', how='left')
escolas_quantidade_md.drop(columns=['nome'], inplace=True)
escolas_quantidade_md.head(2)

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,tipo,complemento,numero_rua,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,880888","-43,225326",20,CIEP,SEM COMPLEMENTO,0,CENTRO
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,"-22,889574","-43,236202",121,EM,SEM COMPLEMENTO,187,CENTRO


### Ajustando casas decimais de latitude e longitude

In [43]:
escolas_quantidade_md['lat'] = escolas_quantidade_md['lat'].str[:9]
escolas_quantidade_md['lon'] = escolas_quantidade_md['lon'].str[:9]
escolas_quantidade_md.head(2)

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,tipo,complemento,numero_rua,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,88088","-43,22532",20,CIEP,SEM COMPLEMENTO,0,CENTRO
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,"-22,88957","-43,23620",121,EM,SEM COMPLEMENTO,187,CENTRO


In [44]:
# Convertendo id para str
# escolas_quantidade_md['id'] = escolas_quantidade_md['id'].astype('str')
# # Convertendo quantidade para int
# escolas_quantidade_md['quantidade'] = escolas_quantidade_md['quantidade'].str.replace("'", "", regex=True)

# # Substituir 'NAN' por 0
# escolas_quantidade_md['quantidade'] = escolas_quantidade_md['quantidade'].replace('NAN', '0')

# # Substituir linhas com apenas aspas por 0
# escolas_quantidade_md['quantidade'] = escolas_quantidade_md['quantidade'].replace('', '0')
# # Substituir linhas com apenas aspas por 0
# escolas_quantidade_md['quantidade'] = escolas_quantidade_md['quantidade'].replace(' ', '0')

# # Converter a coluna 'quantidade' para o tipo numérico, se necessário
# escolas_quantidade_md['quantidade'] = pd.to_numeric(escolas_quantidade_md['quantidade'])
# escolas_quantidade_md.info()

#############
# #Alternativa:
 # Convertendo a coluna Quantidade para o tipo númérico. Os que não puderem ser convertidos, serão NaN.
escolas_quantidade_md['quantidade'] = pd.to_numeric(escolas_quantidade_md['quantidade'], errors='coerce')
#Excluir as colunas com NaN
escolas_quantidade_md.dropna(subset=['quantidade'], inplace=True)
# Convertendo a coluna Quantidade para int
escolas_quantidade_md['quantidade'] = escolas_quantidade_md['quantidade'].astype(int)
#Coluna id sempre com 3 dígitos
escolas_quantidade_md['id'] = escolas_quantidade_md['id'].apply(lambda x: f'{x:03d}') 
# escolas_quantidade_md.info()
# print(escolas_quantidade_md)
escolas_quantidade_md.head()

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,tipo,complemento,numero_rua,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,88088","-43,22532",20,CIEP,SEM COMPLEMENTO,0,CENTRO
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,"-22,88957","-43,23620",121,EM,SEM COMPLEMENTO,187,CENTRO
2,483,EM CELESTINO SILVA,CENTRO,RUA DO LAVRADIO,"-22,90929","-43,18357",220,EM,SEM COMPLEMENTO,56,CENTRO
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA,"-22,89762","-43,22745",190,ESCOLA,SEM COMPLEMENTO,20,CENTRO
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN,"-22,92441","-43,20857",67,EM,SEM COMPLEMENTO,45,CENTRO


### Material a ser entregue

In [45]:
# um arquivo csv no qual as linhas já estarão ordenas de acordo com a rota a ser seguida.
escolas_quantidade_md.to_csv('rota_a_ser_seguida.csv',index=False)
escolas_quantidade_md.head(2)

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,tipo,complemento,numero_rua,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,88088","-43,22532",20,CIEP,SEM COMPLEMENTO,0,CENTRO
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,"-22,88957","-43,23620",121,EM,SEM COMPLEMENTO,187,CENTRO


In [46]:
# Agrupe por subprefeitura e some a coluna 'quantidade'
material_por_subprefeitura = escolas_quantidade_md.groupby('subprefeitura')['quantidade'].sum().reset_index()
material_por_subprefeitura.to_csv('material_por_subprefeitura.csv', index=False)
material_por_subprefeitura.head(2)

,subprefeitura,quantidade
0,BARRA DA TIJUCA,215
1,CENTRO,2352
